# <span style="color:green"><center>Diplomado en Big Data</center></span>

# <span style="color:red"><center>Aprendizaje de máquinas en Paralelo y distribuido <center></span>

<img src="../images/dask_horizontal.svg" align="right" width="30%">


##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Contenido</span>


* [Introducción](#Introducción)
* [Scikit-Learn en 5 minutos](#Scikit-Learn-en-5-minutos)
* [Hiperparámetros](#Hiperparámetros)
* [Optimización de hiperparámetros](#Optimización-de-hiperparámetros)
* [Paralelización con una sola máquina y scikit-learn](#Paralelización-con-una-sola-máquina-y-scikit-learn)
* [Paralelización con  varias máquinas con Dask](#Paralelización-con-varias-máquinas-y-Dask)
* [Entrenamiento sobre grandes conjuntos de datos](#Entrenamiento-sobre-grandes-conjuntos-de-datos)


## <span style="color:blue">Fuente</span>

Esta es una traducción libre del tutorial disponible en [dask-tutorial](https://github.com/dask/dask-tutorial).

## <span style="color:blue">Introducción</span>

[Dask-ML](https://dask-ml.readthedocs.io) tiene recursos para aprendizaje de máquinas (`machine learning`).

Un panorama de la arquitectura de  trabajo de big data con Python y Dask, o apreciamos en esta imagen
![](../images/architecture-1536x947.png)


Hay un par de problemas de escala distintos a los que podemos enfrentarnos.

La estrategia de escala depende del problema que se enfrente.

1. Vinculado a la CPU: los datos caben en la RAM, pero el entrenamiento lleva demasiado tiempo. Muchas combinaciones de hiperparámetros, un gran conjunto de muchos modelos, etc.
2. Límite de memoria: los datos son más grandes que la RAM y el muestreo no es una opción.

![](../images/ml-dimensions.png)

* Para problemas en la memoria, simplemente use scikit-learn (o su biblioteca ML favorita).
* Para modelos grandes, use `dask_ml.joblib` y su estimador scikit-learn favorito
* Para conjuntos de datos grandes, use estimadores `dask_ml`

## <span style="color:blue">Scikit-Learn en 5 minutos</span>

Scikit-Learn tiene una API agradable y consistente.

1. usted crea una instancia de un `Estimator` (por ejemplo,` LinearRegression`, `RandomForestClassifier`, etc.). Todos los modelos *hiperparámetros* (parámetros especificados por el usuario, no los aprendidos por el estimador) se pasan al estimador cuando se crea.
2. Se llama *estimator.fit (X, y)* para entrenar al estimador.
3. Utilice el *estimador* para inspeccionar atributos, hacer predicciones, etc.

### Generemos algunos datos aleatoriamente

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=4, random_state=0)
X[:8]

In [ ]:
y[:8]


Vamos a ajustar un clasificador de soprte de vectores(support vector machine, `SVM`)

In [ ]:
from sklearn.svm import SVC

Creamos un estimados y lo ajustamos

In [ ]:
estimator = SVC(random_state=0)
estimator.fit(X, y)

Inspeccionamos los atributos aprendidos.

In [ ]:
estimator.support_vectors_[:4]

Chequemos la exactitud.

In [ ]:
estimator.score(X, y)

## <span style="color:blue">Hiperparámetros</span>

La mayoría de los modelos tienen *hiperparámetros*. Estos afectan al ajuste, pero se especifican por adelantado en lugar de aprenderlos durante el entrenamiento.

In [ ]:
estimator = SVC(C=0.00001, shrinking=False, random_state=0)
estimator.fit(X, y)
estimator.support_vectors_[:4]

In [ ]:
estimator.score(X, y)

## <span style="color:blue">Optimización de hiperparámetros</span>


Hay algunas formas de aprender los mejores * hiper * parámetros durante el entrenamiento. Uno es "GridSearchCV".
Como su nombre lo indica, esto hace una búsqueda de fuerza bruta sobre una cuadrícula de combinaciones de hiperparámetros.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2)
grid_search.fit(X, y)

## <span style="color:blue">Paralelización con una sola máquina y scikit-learn</span>


Scikit-Learn tiene un buen paralelismo  con *una sola máquina*, a través de `Joblib`.
Cualquier estimador de scikit-learn que pueda operar en paralelo expone una palabra clave `n_jobs`.
Esto controla la cantidad de núcleos de CPU que se utilizarán.![](../images/unmerged_grid_search_graph.svg)


In [ ]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

## <span style="color:blue">Paralelización con varias máquinas y Dask</span>


![](../images/merged_grid_search_graph.svg)

Dask puede hablar con scikit-learn (a través de joblib) para que su *clúster* se use para entrenar un modelo.

Si ejecuta esto en una computadora portátil, tomará bastante tiempo, pero el uso de la CPU será satisfactoriamente cercano al 100% mientras dure. Para ejecutar más rápido, necesitaría un clúster distribuido. Eso significaría poner algo en la llamada a "Cliente" algo como

```
c = Client('tcp://my.scheduler.address:8786')
```

Se pueden encontrar detalles sobre las muchas formas de crear un clúster. [aquí](https://docs.dask.org/en/latest/setup/single-distributed.html).

También puede usar un cluster de `Coiled`, `Saturn`, etc, como vimos en la lección anterior

Probémos en un problema mayor (más hiperparámetros).

In [ ]:
import joblib
import dask.distributed

c = dask.distributed.Client()

In [ ]:
param_grid = {
    'C': [0.001, 0.1, 1.0, 2.5, 5, 10.0],
    # Uncomment this for larger Grid searches on a cluster
    # 'kernel': ['rbf', 'poly', 'linear'],
    # 'shrinking': [True, False],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=5, n_jobs=-1)

In [ ]:
%%time
with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

In [ ]:
grid_search.best_params_, grid_search.best_score_

## <span style="color:blue">Entrenamiento sobre grandes conjuntos de datos</span>


A veces querrá entrenar en un conjunto de datos más grande que la memoria. `dask-ml` ha implementado estimadores que funcionan bien con  `dask.array` y  `dask.dataframes` que pueden ser más grandes que la RAM de su máquina.

In [ ]:
import dask.array as da
import dask.delayed
from sklearn.datasets import make_blobs
import numpy as np

Crearemos un pequeño conjunto de datos (aleatorio) localmente usando scikit-learn. `make_blobs`crea muestras de distribuciones Gaussianas con distintos centros. Para entrenamiento de clasificadores.

In [ ]:
n_centers = 12
n_features = 20

X_small, y_small = make_blobs(n_samples=1000, centers=n_centers, n_features=n_features, random_state=0)

centers = np.zeros((n_centers, n_features))

for i in range(n_centers):
    centers[i] = X_small[y_small == i].mean(0)
    
centers[:4]

El pequeño conjunto de datos será la plantilla para nuestro gran conjunto de datos aleatorios.
Usaremos `dask.delayed` para adaptar` sklearn.datasets.make_blobs`, de modo que el conjunto de datos real se genere en nuestros trabajadores.

In [ ]:
n_samples_per_block = 200000
n_blocks = 500

delayeds = [dask.delayed(make_blobs)(n_samples=n_samples_per_block,
                                     centers=centers,
                                     n_features=n_features,
                                     random_state=i)[0]
            for i in range(n_blocks)]
arrays = [da.from_delayed(obj, shape=(n_samples_per_block, n_features), dtype=X.dtype)
          for obj in delayeds]
X = da.concatenate(arrays)
X

In [ ]:
X = X.persist()  # Only run this on the cluster.

Los algoritmos implementados en Dask-ML son escalables. Manejan bien conjuntos de datos más grandes que la memoria.

Siguen la API scikit-learn, por lo que si está familiarizado con scikit-learn, se sentirá como en casa con Dask-ML.

In [ ]:
from dask_ml.cluster import KMeans

In [ ]:
clf = KMeans(init_max_iter=3, oversampling_factor=10)

In [ ]:
%time clf.fit(X)

In [ ]:
clf.labels_

In [ ]:
clf.labels_[:10].compute()